In [ ]:
import time, network, espnow
from machine import Pin, PWM

MAC_EMISOR = b'\x6C\xC8\x40\x59\x16\x98' #  MAC del emisor

# ---------- Pines L298N (IGUAL) ----------
IN1 = Pin(27, Pin.OUT)  # Motor izquierdo IN1
IN2 = Pin(14, Pin.OUT)  # Motor izquierdo IN2
IN3 = Pin(12, Pin.OUT)  # Motor derecho IN3
IN4 = Pin(13, Pin.OUT)  # Motor derecho IN4

ENA = PWM(Pin(4), freq=500)   # PWM izquierdo
ENB = PWM(Pin(5), freq=500)   # PWM derecho

# Inicializar apagados
ENA.duty(0)
ENB.duty(0)
IN1.off(); IN2.off(); IN3.off(); IN4.off()

MAX_DUTY = 1023
LED = Pin(2, Pin.OUT)

# Control Difuso
class ControlDifusoMotor:
    def __init__(self, motor_name):
        self.activo = False
        self.comando = "NEUTRAL"
        self.vel_objetivo = 0
        self.vel_actual = 0
        self.vel_base = 60

        # PARÁMETROS
        self.MIN_DUTY = 380
        self.MAX_DUTY = 850

        # Factores específicos por motor
        if motor_name == "LEFT":
            self.factor_correccion = 0.98
        else:  # RIGHT
            self.factor_correccion = 1.02

        # Historial para suavizado
        self.historial = [0, 0, 0]

    def detener_motor(self, pwm_pin, in1_pin, in2_pin):
        """Detener motor suavemente"""
        while self.vel_actual > 50:
            self.vel_actual -= 30
            if self.vel_actual < 0:
                self.vel_actual = 0
            pwm_pin.duty(int(self.vel_actual))
            time.sleep(0.02)

        pwm_pin.duty(0)
        in1_pin.off(); in2_pin.off()
        self.activo = False
        self.comando = "NEUTRAL"
        self.vel_objetivo = 0
        self.vel_actual = 0

    def calcular_velocidad(self, percent):
        """Calcular velocidad con mínimo funcional (IGUAL)"""
        duty = int(MAX_DUTY * (percent / 100.0))

        if duty > 0:
            duty = max(self.MIN_DUTY, duty)
            duty = min(duty, self.MAX_DUTY)

        return duty

    def configurar_movimiento(self, cmd, speed_percent=None, pwm_pin=None, in1_pin=None, in2_pin=None):
        """Configurar movimiento específico para cada motor"""
        if speed_percent is None:
            speed_percent = self.vel_base

        self.vel_objetivo = self.calcular_velocidad(speed_percent)
        self.comando = cmd
        self.activo = True

        # Configurar dirección
        if cmd == "ADELANTE":
            in1_pin.on(); in2_pin.off()
        elif cmd == "ATRAS":
            in1_pin.off(); in2_pin.on()
        elif cmd == "IZQUIERDA":
            if "LEFT" in str(self):
                in1_pin.off(); in2_pin.off()  # Izquierdo neutral
            else:
                in1_pin.on(); in2_pin.off()   # Derecho adelante
            self.vel_objetivo = int(self.vel_objetivo * 0.7)
        elif cmd == "DERECHA":
            if "LEFT" in str(self):
                in1_pin.on(); in2_pin.off()   # Izquierdo adelante
            else:
                in1_pin.off(); in2_pin.off()  # Derecho neutral
            self.vel_objetivo = int(self.vel_objetivo * 0.7)
        elif cmd in ["NEUTRAL", "OFF", "STOP"]:
            self.detener_motor(pwm_pin, in1_pin, in2_pin)
            return

    def aplicar_control_difuso(self, pwm_pin):
        """Aplicar control difuso - ADAPTADO PARA CADA MOTOR"""
        if not self.activo or self.comando == "NEUTRAL":
            return

        # Rampa hacia velocidad objetivo
        diff = self.vel_objetivo - self.vel_actual

        if abs(diff) < 10:
            step = 2
        elif abs(diff) < 50:
            step = 5
        elif abs(diff) < 150:
            step = 15
        else:
            step = 25

        if diff > 0:
            self.vel_actual += step
            if self.vel_actual > self.vel_objetivo:
                self.vel_actual = self.vel_objetivo
        else:
            self.vel_actual -= step
            if self.vel_actual < 0:
                self.vel_actual = 0

        # Calcular factores según movimiento
        if self.comando in ["ADELANTE", "ATRAS"]:
            velocidad_ajustada = int(self.vel_actual * self.factor_correccion)
        else:
            velocidad_ajustada = self.vel_actual

        # Aplicar mínimo funcional
        if velocidad_ajustada > 0 and velocidad_ajustada < self.MIN_DUTY:
            velocidad_ajustada = self.MIN_DUTY

        # Limitar máximo
        velocidad_ajustada = min(velocidad_ajustada, self.MAX_DUTY)

        # Suavizar con historial
        self.historial.pop(0)
        self.historial.append(velocidad_ajustada)
        velocidad_suave = sum(self.historial) // len(self.historial)

        # APLICAR a motor específico
        pwm_pin.duty(velocidad_suave)

# Inicializar motores
print("\n" + "="*60)
print("RECEPTOR - 2 Motores Independientes")
print("="*60)

motor_left = ControlDifusoMotor("LEFT")
motor_right = ControlDifusoMotor("RIGHT")

w = network.WLAN(network.STA_IF)
w.active(True)
e = espnow.ESPNow()
e.active(True)

try:
    e.add_peer(MAC_EMISOR)
    print(f"Conectado al emisor: {MAC_EMISOR.hex()}")
except Exception as err:
    print(f"Error conectando: {err}")

print("\nSistema listo. Esperando comandos...")

# Variables de control
ultimo_mensaje = time.ticks_ms()
TIEMPO_TIMEOUT = 800
recibiendo_señal = True

print(f"Timeout: {TIEMPO_TIMEOUT}ms sin señal = detener")

while True:
    tiempo_actual = time.ticks_ms()

    try:
        r = e.recv(timeout=0.05)
    except:
        try:
            r = e.recv()
        except:
            r = None

    if r:
        try:
            _, msg = r
        except:
            try:
                _, msg = r[0], r[1]
            except:
                msg = None

        if msg:
            ultimo_mensaje = tiempo_actual
            recibiendo_señal = True

            LED.on()
            time.sleep(0.005)
            LED.off()

            try:
                texto = msg.decode('utf-8')
            except:
                texto = None

            if texto:
                print(f"RX: {texto}")

                if texto == "SYNC":
                    pass

                elif texto == "confirm":
                    motor_left.configurar_movimiento("ADELANTE", 40, ENA, IN1, IN2)
                    motor_right.configurar_movimiento("ADELANTE", 40, ENB, IN3, IN4)
                    time.sleep(0.3)
                    motor_left.detener_motor(ENA, IN1, IN2)
                    motor_right.detener_motor(ENB, IN3, IN4)

                elif texto.startswith("SPEED:"):
                    try:
                        vel = int(texto.split(":")[1])
                        motor_left.vel_base = max(10, min(100, vel))
                        motor_right.vel_base = max(10, min(100, vel))
                        print(f"Velocidad base: {motor_left.vel_base}%")
                    except:
                        pass

                elif texto.startswith("LEFT:") and "|RIGHT:" in texto:
                    partes = texto.split("|")

                    left_parts = partes[0].split(":")
                    if len(left_parts) >= 3:
                        left_cmd = left_parts[1].upper()
                        try:
                            left_vel = int(left_parts[2])
                            motor_left.configurar_movimiento(left_cmd, left_vel, ENA, IN1, IN2)
                        except:
                            motor_left.configurar_movimiento(left_cmd, None, ENA, IN1, IN2)

                    right_parts = partes[1].split(":")
                    if len(right_parts) >= 3:
                        right_cmd = right_parts[1].upper()
                        try:
                            right_vel = int(right_parts[2])
                            motor_right.configurar_movimiento(right_cmd, right_vel, ENB, IN3, IN4)
                        except:
                            motor_right.configurar_movimiento(right_cmd, None, ENB, IN3, IN4)

                    print(f"LEFT: {left_cmd} | RIGHT: {right_cmd}")

                elif cmd in ["OFF", "STOP", "NEUTRAL"]:
                    print(">> Comando de detención")
                    motor_left.detener_motor(ENA, IN1, IN2)
                    motor_right.detener_motor(ENB, IN3, IN4)
                    recibiendo_señal = False

    # VERIFICAR TIMEOUT
    tiempo_sin_señal = time.ticks_diff(tiempo_actual, ultimo_mensaje)

    if tiempo_sin_señal > TIEMPO_TIMEOUT:
        if (motor_left.activo or motor_right.activo) and recibiendo_señal:
            print(f">> SIN SEÑAL ({tiempo_sin_señal}ms) - DETENIENDO")
            motor_left.detener_motor(ENA, IN1, IN2)
            motor_right.detener_motor(ENB, IN3, IN4)
            recibiendo_señal = False

        # LED de espera
        if (tiempo_actual // 1000) % 2 == 0:
            LED.on()
        else:
            LED.off()
    else:
        if recibiendo_señal:
            LED.off()

    # APLICAR CONTROL DIFUSO A MOTORES
    if motor_left.activo:
        motor_left.aplicar_control_difuso(ENA)
    if motor_right.activo:
        motor_right.aplicar_control_difuso(ENB)

    time.sleep(0.03)